<figure>
  <IMG SRC="https://raw.githubusercontent.com/fmeer/public-files/main/TUlogo.png" WIDTH=200 ALIGN="right">
</figure>

# CIEM5110-2 Workshop 6: Nonlinear FEM with plasticity (notebook 2)

In [ ]:
import sys
sys.path.append('../../')

import numpy as np
import matplotlib.pyplot as plt
import contextlib

from utils import proputils as pu
from names import GlobNames as gn
import main
import declare

## Plasticity for a more complex 2D domain

In this final demonstration, we look at the following geometry and boundary conditions:

<center><img src="https://raw.githubusercontent.com/ibcmrocha/public/main/5110_multihole.png" alt="mesh" width="800"/></center>

i.e. we pull this square-shaped domain on the right edge while keeping the left edge fixed horizontally and the bottom edge fixed vertically.

Here we use the $J_2$ plasticity model from before but now in 2D under plane stress assumptions. The model assumes the following yield surface:

$$f = \sqrt{3J_2} + \sigma_\mathrm{y}$$

where the yield stress $\sigma_\mathrm{y}$ is modeled with an <span style="color:red">**exponential hardening**</span> behavior:

$$ \sigma_\mathrm{y} = 64.8 - 33.6\exp\left(\frac{\kappa}{-0.003407}\right)$$

The simulation is run in <span style="color:red">**displacement control**</span>. To allow for a more complex behavior to be investigated, we let the displacement of the right edge of the model (`props['model']['dispcontrol']['timeSignal']`) change in time according to:

$$ \mathtt{dx}\left(t\right) = \frac{1}{2}\sin\left(2\pi t\right)$$

with $\Delta t=0.01\,\mathrm{s}$ for $100$ steps (i.e. a total pseudo-time of $1\,\mathrm{s}$). We then use `LoadDispModule` and `GraphModule` to plot the load-displacement curve of the right edge of the model.

In the file `voids.pro` you can find an initial version of the properties file for this problem. You will also find <span style="color:red">***`???` marks at several places***</span>. Fill them in correctly and run the model with the block below.

In [ ]:
props = pu.parse_file('voids.pro')

globdat = main.jive(props)

From `J2Material`, we store in `globdat` the maximum value of $\kappa$ throughout the whole domain $\Omega$ after every time step. We can then plot this to see how $\kappa$ is evolving in time:

In [ ]:
plt.figure()
plt.xlabel('Time step')
plt.ylabel('$\max_\Omega(\kappa)$')
plt.plot(range(len(globdat['maxkappa'])),globdat['maxkappa'])

For this model, $\kappa$ is an accumulated measure of plastic strain given by:

$$\kappa(t)=\displaystyle\int_0^t\sqrt{\frac{2}{3}\dot{\boldsymbol{\varepsilon}}^\mathrm{p}(t):\dot{\boldsymbol{\varepsilon}}^\mathrm{p}(t)}\,dt$$

With this in mind, do the results above make sense?

Finally, we can look at the displacement field evolution for this model:

In [ ]:
view = globdat[gn.MODULEFACTORY].get_module('View','view')

props['view'] = {}
props['view']['plot'] = 'solution[dx]'
props['view']['deform'] = 1
props['view']['ncolors'] = 100
props['view']['interactive'] = 'True'
props['view']['colorMap'] = 'plasma_r'
props['view']['step0'] = 50

view.init(props, globdat)
status = view.shutdown(globdat)